In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import datetime
import time
import copy
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

In [2]:
# Fungsi untuk menampilkan teks pada gambar
def DrawText(img, sText, x, y):
    font = cv2.FONT_HERSHEY_SIMPLEX
    posf = (x, y)
    fontScale = 5
    fontColor = (255, 255, 255)
    thickness = 1
    lineType = 2
    cv2.putText(img, sText, posf, font, fontScale, fontColor, thickness, lineType)
    return copy.deepcopy(img)


In [3]:
# Fungsi untuk memuat citra training
def LoadCitraTraining(sDir, LabelKelas):
    JumlahKelas = len(LabelKelas)
    TargetKelas = np.eye(JumlahKelas)

    X = []  # Menampung Data Citra
    T = []  # Menampung Target

    for i in range(len(LabelKelas)):
        DirKelas = os.path.join(sDir, LabelKelas[i])
        files = os.listdir(DirKelas)

        for f in files:
            ff = f.lower()
            if ff.endswith(('.jpg', '.jpeg', '.png')):
                NmFile = os.path.join(DirKelas, f)
                img = np.double(cv2.imread(NmFile, 1))
                img = cv2.resize(img, (128, 128))
                img = np.asarray(img) / 255
                img = img.astype('float32')
                X.append(img)
                T.append(TargetKelas[i])

    X = np.array(X).astype('float32')
    T = np.array(T).astype('float32')
    return X, T

In [4]:
# Fungsi untuk membuat model CNN
def ModelDeepLearningCNN(JumlahKelas):
    input_img = Input(shape=(128, 128, 3))
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(JumlahKelas, activation='softmax')(x)
    
    ModelCNN = Model(input_img, x)
    ModelCNN.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return ModelCNN


In [5]:
# Fungsi untuk melakukan training pada model
def TrainingCNN(JumlahEpoh, DirektoriDataSet, LabelKelas, NamaFileBobot='weight.h5'):
    X, D = LoadCitraTraining(DirektoriDataSet, LabelKelas)
    JumlahKelas = len(LabelKelas)
    ModelCNN = ModelDeepLearningCNN(JumlahKelas)
    history = ModelCNN.fit(X, D, epochs=JumlahEpoh, shuffle=True)
    ModelCNN.save(NamaFileBobot)
    return ModelCNN, history

In [6]:
# Fungsi untuk melakukan klasifikasi pada gambar
def Klasifikasi(Image, ModelCNN):
    X = []
    img = copy.deepcopy(Image)
    img = cv2.resize(img, (128, 128))
    img = np.asarray(img) / 255
    img = img.astype('float32')
    X.append(img)
    X = np.array(X).astype('float32')
    hs = ModelCNN.predict(X, verbose=0)
    if hs.max() > 0.5:
        idx = np.argmax(hs)
    else:
        idx = -1
    return idx

In [7]:
# Fungsi untuk membuat direktori
def GetFileName():
    x = datetime.datetime.now()
    s = x.strftime('%Y-%m-%d-%H%M%S%f')
    return s

def CreateDir(path):
    ls = []
    head_tail = os.path.split(path)
    ls.append(path)
    while len(head_tail[1]) > 0:
        head_tail = os.path.split(path)
        path = head_tail[0]
        ls.append(path)
        head_tail = os.path.split(path)
    for i in range(len(ls) - 2, -1, -1):
        sf = ls[i]
        isExist = os.path.exists(sf)
        if not isExist:
            os.makedirs(sf)


In [10]:
# Fungsi untuk membuat dataset menggunakan kamera
def CreateDataSet(NoKamera, NamaDataSet, DirektoriDataSet="D:\\agung\\datadatadata"):
    DirektoriData = os.path.join(DirektoriDataSet, NamaDataSet, GetFileName())
    CreateDir(DirektoriData)
    
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    imsize = (640, 480)
    height, width = imsize
    TimeStart = time.time()
    FrameRate = 5
    
    cap = cv2.VideoCapture(NoKamera, cv2.CAP_DSHOW)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue
    
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, imsize)
            results = pose.process(image)
            if not results.pose_landmarks:
                continue
    
            lm = []
            for lmark in results.pose_landmarks.landmark:
                if (lmark.x > 0.01) and (lmark.x < 1 - 0.01) and (lmark.y > 0.01) and (lmark.y < 1 - 0.01):
                    m = [lmark.x * width, lmark.y * height]
                    lm.append(m)
            if len(lm) == 0:
                continue
            
            lm = np.array(lm)
            xmin, ymin = np.min(lm, axis=0).astype(int)
            xmax, ymax = np.max(lm, axis=0).astype(int)
    
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            bimage = np.zeros((height, width, 3), np.uint8)
            cv2.rectangle(bimage, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            mp_drawing.draw_landmarks(bimage, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            cropped_image = bimage[ymin:ymax, xmin:xmax, :]
            if time.time() - TimeStart > 1 / FrameRate:
                TimeStart = time.time()
                sFile = os.path.join(DirektoriData, GetFileName())
                cropped_image = cv2.resize(cropped_image, (128, 128))
                cv2.imwrite(sFile + '.jpg', cropped_image)
                cv2.imwrite(sFile + '.png', image)
    
            cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()
    cv2.destroyAllWindows()

In [11]:
# Fungsi untuk menguji pose dari dataset yang telah dibuat
def TesPosePrediction(DirDataSet, DirKlasifikasi, LabelKelas, ModelCNN=[]):
    if not ModelCNN:
        ModelCNN = load_model('weight.h5')
    
    X = []
    DirKelas = os.path.join(DirDataSet, DirKlasifikasi)
    files = os.listdir(DirKelas)
    for f in files:
        ff = f.lower()
        if ff.endswith(('.jpg', '.jpeg', '.png')):
            NmFile = os.path.join(DirKelas, f)
            img = cv2.imread(NmFile, 1)
            img = cv2.resize(img, (128, 128))
            img = np.asarray(img) / 255
            img = img.astype('float32')
            X.append(img)
    
    X = np.array(X).astype('float32')
    hs = ModelCNN.predict(X, verbose=0)
    
    LKelasCitra = []
    n = X.shape[0]
    for i in range(n):
        v = hs[i, :]
        if v.max() > 0.5:
            idx = np.argmax(v)
            LKelasCitra.append(LabelKelas[idx])
        else:
            LKelasCitra.append("-")
    return LKelasCitra

In [12]:
# Fungsi untuk memprediksi pose menggunakan kamera
def PredictPose(NoKamera, LabelKelas):
    ModelCNN = load_model('weight.h5')
    
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    imsize = (640, 480)
    height, width = imsize
    
    cap = cv2.VideoCapture(NoKamera, cv2.CAP_DSHOW)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue
    
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, imsize)
            results = pose.process(image)
            if not results.pose_landmarks:
                continue
    
            lm = []
            for lmark in results.pose_landmarks.landmark:
                if (lmark.x > 0.01) and (lmark.x < 1 - 0.01) and (lmark.y > 0.01) and (lmark.y < 1 - 0.01):
                    m = [lmark.x * width, lmark.y * height]
                    lm.append(m)
            if len(lm) == 0:
                continue
            
            lm = np.array(lm)
            xmin, ymin = np.min(lm, axis=0).astype(int)
            xmax, ymax = np.max(lm, axis=0).astype(int)
    
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            bimage = np.zeros((height, width, 3), np.uint8)
            cv2.rectangle(bimage, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            mp_drawing.draw_landmarks(bimage, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            cropped_image = bimage[ymin:ymax, xmin:xmax, :]
            idx = Klasifikasi(cropped_image, ModelCNN)
            x, y = 60, 60
            image = cv2.flip(image, 1)
    
            if idx >= 0:
                cv2.putText(image, LabelKelas[idx], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255, 255, 0), 3)
    
            cv2.imshow('Prediksi Pose', image)
    
            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()
    cv2.destroyAllWindows()

In [14]:
# Parameter dan eksekusi
DirektoriDataSet = "D:\\agung\\datadatadata"
LabelKelas = ["TanganKiri", "TanganKanan", "Berhenti", "Maju", "Mundur"]

In [15]:
# Membuat Data Set
CreateDataSet(0, "Berhenti", DirektoriDataSet)---------------------------------------------------------------------------------------------------------------------------------------

d:\agung\.venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 

In [ ]:
# Melakukan training model
JumlahEpoh = 10
ModelCNN, history = TrainingCNN(JumlahEpoh, DirektoriDataSet, LabelKelas)

In [ ]:
ModelCNN.summary()

In [ ]:
# Menampilkan grafik loss
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
# Memprediksi pose menggunakan kamera
PredictPose(0, LabelKelas)